In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv("us.env"))  # read local .env file

In [2]:
from pathlib import Path
from tqdm import tqdm
import sys

sys.path.append(Path("..").resolve().as_posix())

## Prepare LLMs

In [3]:
from src.generator_builder import LLMConfig

llms = LLMConfig.from_azure_deployments(
    light="gpt-35-turbo-1106",
    heavy="gpt-4-1106",
    embedding="text-embedding-ada-002",
    api_version="2023-07-01-preview",
)
from llama_index.core import Settings

Settings.embed_model = llms.embed_model
Settings.llm = llms.llamaindex_light

## Assemble the RAG

In [4]:
from src.index_builder import build_index

index, full_nodes_dict = build_index(
    persist_path = Path("index_storage").resolve(),
    collection_name = "dev1",
    lib_path = Path("../../docs_md").resolve()
)

In [5]:
from src.generator_builder import build_generator

generator = build_generator(
    index=index, full_nodes_dict=full_nodes_dict, llm_config=llms
)

In [6]:
response = generator.query("Tell me about a basic select")
print(response.response)

A basic `select` in EdgeQL is used to retrieve or compute a set of values from the database. It can be as simple as selecting primitive values or more complex when selecting objects with various properties and relationships. Here's an example of a basic `select` statement that retrieves primitive values: ```edgeql-repl db> select 'hello world'; {'hello world'} db> select [1, 2, 3]; {[1, 2, 3]} db> select {1, 2, 3}; {1, 2, 3} ``` In the context of a database schema, a `select` statement is often used to retrieve objects. For example, consider the following schema: ```sdl module default { type Person { required property name -> str; multi link friends -> Person; } } ``` Given this schema, you can select all `Person` objects with their names like this: ```edgeql-repl db> select Person { name }; ``` This will return a set of `Person` objects with their `name` property. By default, only the `id` of each object is returned, but by using a shape (enclosed in curly braces `{}`), you can specif

## Run benchmarks

In [ ]:
easy_query_path = Path("benchmarks/tutorial_benchmark.txt").resolve()
# hard_query_path = Path("benchmarks/user_benchmark.txt").resolve()
hard_query_path = Path("benchmarks/new_hard.txt").resolve()

In [ ]:
with easy_query_path.open("r") as f:
    easy_queries = [query.strip() for query in f.readlines()]

with hard_query_path.open("r") as f:
    hard_queries = f.read().split(">>>SEPARATOR<<<")

In [ ]:
len(hard_queries)

In [ ]:
from src.eval import PydanticResponse, run_queries

In [ ]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
responses = run_queries(query_engine, hard_queries[147:], Path(f"eval_new_hard_{timestamp}.jsonl").resolve())